In [1]:
import os
from pathlib import Path
from datetime import datetime, date

from pyspark.conf import SparkConf
from pyspark.sql import Row, SparkSession

In [2]:
spark_conf = SparkConf()
spark_conf.setAll([
    ("spark.app.name", "Delta Lake"),
    ("spark.ui.showConsoleProgress", "true"),
    ("spark.eventLog.enabled", "false"),
    # see https://docs.delta.io/latest/quick-start.html#pyspark-shell
    ("spark.jars", "file://{}".format(os.path.join(Path().resolve(), "..", "jars", "delta-core_2.12-1.0.0.jar"))),
    ("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension"),
    ("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
])

In [3]:
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

21/08/22 07:43:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df.groupBy('a').avg().show()

+---+------+------+
|  a|avg(a)|avg(b)|
+---+------+------+
|  1|   1.0|   2.0|
|  2|   2.0|   3.0|
|  4|   4.0|   5.0|
+---+------+------+



In [9]:
table_dir = "/workspaces/workspace-lakehouse/delta-lake/tmp/delta-table"

if os.path.exists(table_dir) and os.path.isdir(table_dir):
    import shutil
    shutil.rmtree(table_dir)

data = spark.range(0, 5)
data.write.format("delta").save(table_dir)

In [10]:
df = spark.read.format("delta").load(table_dir)
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  0|
|  1|
|  4|
+---+



In [11]:
spark.sql("SELECT * FROM delta.`{}`".format(table_dir)).count()

5

In [12]:
spark.sparkContext._gateway.close()
spark.stop()